In [ ]:
# @title getting data1
import zipfile
from PIL import Image
import json
import io

def load_from_zip(zip_filename):
    with zipfile.ZipFile(zip_filename, 'r') as myzip:
        images = []
        # Iterate over each file in the ZIP
        for filename in myzip.namelist():
            if filename.endswith('_original.png') or filename.endswith('_transformed.png'):
                # Extract the image
                image_data = myzip.read(filename)
                image = Image.open(io.BytesIO(image_data))
                image.load()  # Load image data

                # Extract corresponding metadata
                base_name = filename.rsplit('_', 1)[0]  # Remove the last part after '_'
                metadata_file = f"{base_name}_data.json"
                metadata_data = myzip.read(metadata_file)
                metadata = json.loads(metadata_data)

                # Reconstruct the tuple format
                if '_original.png' in filename:
                    type_img = 'original'
                else:
                    type_img = 'transformed'

                images.append((image, metadata['response'], type_img, metadata['emotion'], metadata['description']))
        return images

# Example usage
zip_filename = 'output_images.zip'
recovered_images = load_from_zip(zip_filename)

images = []
data = []

for img, response, type_img, emotion, description in recovered_images:
    # print(f"Type: {type_img}, Emotion: {emotion}, Description: {description}, Response: {response}")
    # display(img)  # This will display the image
    images.append((img, response, type_img, emotion, description))
    data.append((response, emotion))



In [ ]:
# @title getting data2

# @title json implementation 2 (good implementation)

import json

# Function to load the JSON data into a dictionary
def load_data(json_path):
    with open(json_path, 'r') as file:
        data = json.load(file)
    # Create a dictionary to organize data by file_name
    organized_data = {}
    for item in data:
        filename = item['file_name']
        if filename not in organized_data:
            organized_data[filename] = {
                'emotions': [],
                'descriptions': [],
                'responses': []
            }
        # Append each emotion, description, and response to the corresponding lists
        organized_data[filename]['emotions'].append(item['emotion'])
        organized_data[filename]['descriptions'].append(item['description'])
        organized_data[filename]['responses'].append(item['response'])
    return organized_data

# Function to get specific information by file_name
def get_attributes_by_filename(data, file_name):
    # Check if the file_name exists in the data
    if file_name in data:
        return data[file_name]
    else:
        return "No data found for this file name."

# Load the data
json_path = '/content/output_long.json'
dumbdata = load_data(json_path)

# # Example usage: Retrieving information for a specific file_name
# file_name = 's_05_e_23_shot_028963_029037'  # Change this to the file_name you want to query

# # Get attributes
# attributes = get_attributes_by_filename(data, file_name)

# # Print results
# if isinstance(attributes, dict):
#     print("Emotions:", attributes['emotions'])
#     print("Descriptions:", attributes['descriptions'])
#     print("Responses:", attributes['responses'])
# # else:
# #     print(attributes)
# print("Responses:", attributes['responses'][0])

counter = 0
data2 = []
print(len(dumbdata))
for filename in dumbdata:
    counter += 1
    attributes = get_attributes_by_filename(dumbdata, filename)
    # Print results
    # print(counter)
    if isinstance(attributes, dict):
        if counter >= len(dumbdata):
            break
        for i in range(0, 4):
            # print("Emotions:", attributes['emotions'])
            # print("Descriptions:", attributes['descriptions'])
            # print("Responses:", attributes['responses'])
            # print(i)
            data2.append((attributes['responses'][i], attributes['emotions'][i]))


2776


In [ ]:
data2[1]

('Add wilted flowers on the table next to Barney.', 'sadness')

In [ ]:
from transformers import pipeline
import numpy as np


In [ ]:
# Step 2: Load the emotion classification model
model_name = 'bhadresh-savani/distilbert-base-uncased-emotion'
emotion_classifier = pipeline('text-classification', model=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Define a mapping from model outputs to your target categories
emotion_mapping = {
    'joy': 'happiness',
    'love': 'sadness',  # Assuming you want to ignore or handle 'love' differently
    'anger': 'anger',
    'fear': 'fear',
    'sadness': 'sadness',
    'surprise' : 'anger'
}


In [ ]:
# @title data1 (short) results

# Step 3: Classify each instruction and collect predictions
predictions = []
for text, actual_label in data:
    prediction = emotion_classifier(text)
    # predicted_label = prediction[0]['label'].lower()
    predicted_label = prediction[0]['label']
    mapped_label = emotion_mapping.get(predicted_label, None)
    if mapped_label is None:
        print(f"Warning: Unknown label '{predicted_label}'")
    predictions.append((mapped_label, actual_label))
    # if mapped_label != actual_label:
    #     print(f"og = {predicted_label} \t\tPrediction: {mapped_label}, \t\t Actual Label: {actual_label}")

# Step 4: Calculate accuracy
correct_predictions = sum(1 for predicted, actual in predictions if predicted == actual)
accuracy = correct_predictions / len(data)

print(f"Accuracy: {accuracy:.2f}")
# print("Predictions:", predictions)


Accuracy: 0.60


In [ ]:
# @title data2 (long) results
counter = 0
# Step 3: Classify each instruction and collect predictions
predictions = []
for text, actual_label in data2:
    prediction = emotion_classifier(text)
    # predicted_label = prediction[0]['label'].lower()
    predicted_label = prediction[0]['label']
    mapped_label = emotion_mapping.get(predicted_label, None)
    if mapped_label is None:
        print(f"Warning: Unknown label '{predicted_label}'")
    predictions.append((mapped_label, actual_label))
    # if mapped_label != actual_label:
    #     print(f"og = {predicted_label} \t\tPrediction: {mapped_label}, \t\t Actual Label: {actual_label}")

# Step 4: Calculate accuracy
correct_predictions = sum(1 for predicted, actual in predictions if predicted == actual)
accuracy = correct_predictions / len(data2)

print(f"Accuracy: {accuracy:.2f}")
# print("Predictions:", predictions)


Accuracy: 0.64


In [ ]:
print(predictions)

[('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happiness'), ('happiness', 'happ

# Clip Cosine Similarity

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
def cosine_similarity(tensor1, tensor2):
    """
    Compute the cosine similarity between two tensors.
    """
    tensor1_norm = tensor1 / tensor1.norm(dim=1, keepdim=True)
    tensor2_norm = tensor2 / tensor2.norm(dim=1, keepdim=True)
    return torch.mm(tensor1_norm, tensor2_norm.transpose(0, 1))


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def cosine_similarity(tensor1, tensor2):
    """
    Compute the cosine similarity between two tensors.
    """
    tensor1_norm = tensor1 / tensor1.norm(dim=-1, keepdim=True)
    tensor2_norm = tensor2 / tensor2.norm(dim=-1, keepdim=True)
    return (tensor1_norm * tensor2_norm).sum(dim=-1)

def calculate_similarity(data):
    # Prepare the text for CLIP
    instructions = [entry[0] for entry in data]
    emotions = [entry[1] for entry in data]

    # Process instructions and emotions through CLIP processor
    instruction_inputs = processor(text=instructions, return_tensors="pt", padding=True, truncation=True)
    emotion_inputs = processor(text=emotions, return_tensors="pt", padding=True, truncation=True)

    # Generate embeddings
    with torch.no_grad():
        instruction_embeddings = model.get_text_features(**instruction_inputs)
        emotion_embeddings = model.get_text_features(**emotion_inputs)

    # Calculate cosine similarities for each pair
    similarities = cosine_similarity(instruction_embeddings, emotion_embeddings)
    return similarities

# # Example data
# data = [
#     ("The dog played in the yard", "joy"),
#     ("Look at these beautiful flowers", "admiration")
# ]

# Compute similarities
similarities = calculate_similarity(data)
print("Cosine Similarities between each instruction and corresponding emotion:")
print(similarities)


Cosine Similarities between each instruction and corresponding emotion:
tensor([0.7861, 0.7861, 0.7426, 0.7426, 0.8285, 0.8285, 0.8063, 0.8063, 0.7509,
        0.7509, 0.8382, 0.8382, 0.7900, 0.7900, 0.8137, 0.8137, 0.6874, 0.6874,
        0.7853, 0.7853, 0.8248, 0.8248, 0.7811, 0.7811, 0.7901, 0.7901, 0.7112,
        0.7112, 0.6636, 0.6636, 0.7876, 0.7876, 0.8287, 0.8287, 0.8309, 0.8309,
        0.8264, 0.8264, 0.8106, 0.8106, 0.7721, 0.7721, 0.6600, 0.6600, 0.7823,
        0.7823, 0.7333, 0.7333, 0.7228, 0.7228, 0.7381, 0.7381, 0.8543, 0.8543,
        0.8125, 0.8125, 0.8293, 0.8293, 0.8552, 0.8552, 0.7106, 0.7106, 0.7399,
        0.7399, 0.7656, 0.7656, 0.7617, 0.7617, 0.7508, 0.7508, 0.8315, 0.8315,
        0.7837, 0.7837, 0.6982, 0.6982, 0.8337, 0.8337, 0.8633, 0.8633, 0.7984,
        0.7984, 0.7288, 0.7288, 0.7507, 0.7507, 0.8114, 0.8114, 0.7844, 0.7844,
        0.7380, 0.7380, 0.7928, 0.7928, 0.8157, 0.8157, 0.6709, 0.6709, 0.8161,
        0.8161, 0.6609, 0.6609, 0.8597, 0.8597, 

In [ ]:
# @title data1 (short) cosine similarity

def calculate_similarity(data):
    # Prepare the text for CLIP
    instructions = [entry[0] for entry in data]
    emotions = [entry[1] for entry in data]

    # Process instructions and emotions through CLIP processor
    instruction_inputs = processor(text=instructions, return_tensors="pt", padding=True, truncation=True)
    emotion_inputs = processor(text=emotions, return_tensors="pt", padding=True, truncation=True)

    # Generate embeddings
    with torch.no_grad():
        instruction_embeddings = model.get_text_features(**instruction_inputs)
        emotion_embeddings = model.get_text_features(**emotion_inputs)

    # Calculate cosine similarities for each pair
    similarities = cosine_similarity(instruction_embeddings, emotion_embeddings)
    return similarities, emotions  # Return emotions for grouping


similarities, emotions = calculate_similarity(data)

from collections import defaultdict
import numpy as np

# Initialize dictionary to store sums and counts
emotion_sums = defaultdict(float)
emotion_counts = defaultdict(int)

# Accumulate sums and counts
for similarity, emotion in zip(similarities, emotions):
    emotion_sums[emotion] += similarity.item()  # Convert from tensor to float
    emotion_counts[emotion] += 1

# Calculate averages
emotion_averages = {emotion: emotion_sums[emotion] / emotion_counts[emotion] for emotion in emotion_sums}

print("Average Cosine Similarities by Emotion:")
for emotion, avg_sim in emotion_averages.items():
    print(f"{emotion}: {avg_sim:.3f}")



Average Cosine Similarities by Emotion:
happiness: 0.766
sadness: 0.776
anger: 0.759
fear: 0.803


In [ ]:
total_avg = 0
for emotion, avg_sim in emotion_averages.items():
    # print(f"{emotion}: {avg_sim:.3f}")
    total_avg += avg_sim
total_avg = total_avg / 4

print(total_avg)

0.7760475928846159


In [ ]:
# @title data2 (long) cosine similarity

def calculate_similarity(data):
    # Prepare the text for CLIP
    instructions = [entry[0] for entry in data]
    emotions = [entry[1] for entry in data]

    # Process instructions and emotions through CLIP processor
    instruction_inputs = processor(text=instructions, return_tensors="pt", padding=True, truncation=True)
    emotion_inputs = processor(text=emotions, return_tensors="pt", padding=True, truncation=True)

    # Generate embeddings
    with torch.no_grad():
        instruction_embeddings = model.get_text_features(**instruction_inputs)
        emotion_embeddings = model.get_text_features(**emotion_inputs)

    # Calculate cosine similarities for each pair
    similarities = cosine_similarity(instruction_embeddings, emotion_embeddings)
    return similarities, emotions  # Return emotions for grouping


similarities, emotions = calculate_similarity(data2)

from collections import defaultdict
import numpy as np

# Initialize dictionary to store sums and counts
emotion_sums = defaultdict(float)
emotion_counts = defaultdict(int)

# Accumulate sums and counts
for similarity, emotion in zip(similarities, emotions):
    emotion_sums[emotion] += similarity.item()  # Convert from tensor to float
    emotion_counts[emotion] += 1

# Calculate averages
emotion_averages = {emotion: emotion_sums[emotion] / emotion_counts[emotion] for emotion in emotion_sums}

print("Average Cosine Similarities by Emotion:")
for emotion, avg_sim in emotion_averages.items():
    print(f"{emotion}: {avg_sim:.3f}")



Average Cosine Similarities by Emotion:
happiness: 0.725
sadness: 0.788
anger: 0.770
fear: 0.794


In [ ]:
total_avg = 0
for emotion, avg_sim in emotion_averages.items():
    # print(f"{emotion}: {avg_sim:.3f}")
    total_avg += avg_sim
total_avg = total_avg / 4

print(total_avg)

0.7691745380587407
